In [3]:
import os, numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

WAREHOUSE_DIR = "data_warehouse"
RAW_DIR = "raw_data"
os.makedirs(WAREHOUSE_DIR, exist_ok=True)
os.makedirs("reverse_etl", exist_ok=True)

POSSIBLE_SOURCES = [
    os.path.join(WAREHOUSE_DIR, "processed_sales_data.csv"),
    "processed_sales_data.csv",
    os.path.join(RAW_DIR, "sale_price.csv"),
]
src_path = next((p for p in POSSIBLE_SOURCES if os.path.exists(p)), None)
if src_path is None:
    raise FileNotFoundError(f"Could not find any of: {POSSIBLE_SOURCES}")

print(f"Loading: {src_path}")
df = pd.read_csv(src_path)


if "customer_id" not in df.columns:
    raise ValueError("Expected a 'customer_id' column.")

if "total_revenue" not in df.columns:
    if "sale_price" in df.columns:
        x = df["sale_price"].astype(str).str.replace(r"[\$,]", "", regex=True)
        df["sale_price"] = pd.to_numeric(x, errors="coerce")
    if "quantity" not in df.columns:
        df["quantity"] = 1
    df["total_revenue"] = df["sale_price"].fillna(0) * df["quantity"]


if "sale_date" in df.columns:
    df["sale_date"] = pd.to_datetime(
        df["sale_date"], errors="coerce", dayfirst=True, utc=True
    )
else:
    df["sale_date"] = pd.NaT


g = (
    df.groupby("customer_id", dropna=False)
      .agg(total_spend=("total_revenue", "sum"),
           purchase_count=("total_revenue", "size"),
           last_purchase_date=("sale_date", "max"))
      .reset_index()
)
g["avg_order_value"] = g["total_spend"] / g["purchase_count"].replace(0, np.nan)


today = pd.Timestamp.now(tz="UTC").normalize()
g["recency_days"] = (today - g["last_purchase_date"]).dt.days

if g["recency_days"].isna().all():
    g["recency_days"] = 0
else:
    g["recency_days"] = g["recency_days"].fillna(g["recency_days"].median())

for c in ["total_spend", "purchase_count", "avg_order_value", "recency_days"]:
    g[c] = g[c].fillna(0)


feat_cols = ["total_spend", "purchase_count", "avg_order_value", "recency_days"]
X = StandardScaler().fit_transform(g[feat_cols].to_numpy())
kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
g["cluster"] = kmeans.fit_predict(X)

vip_cluster = g.groupby("cluster")["total_spend"].mean().idxmax()
g["vip_status"] = np.where(g["cluster"] == vip_cluster, "VIP", "Non-VIP")

q80 = g["total_spend"].quantile(0.80)
g["vip_top20pct_by_spend"] = (g["total_spend"] >= q80).astype(int)


enriched = df.merge(
    g[["customer_id", "vip_status", "vip_top20pct_by_spend"]],
    on="customer_id", how="left"
)

row_out  = os.path.join(WAREHOUSE_DIR, "vip_sales_data.csv")
cust_out = os.path.join("reverse_etl", "vip_customers.csv")
enriched.to_csv(row_out, index=False)
g.sort_values("total_spend", ascending=False).to_csv(cust_out, index=False)

print("VIP cluster chosen:", vip_cluster)
print("Written:\n -", row_out, "\n -", cust_out)

peek_cols = [c for c in ["customer_id","product_id","sale_date","total_revenue","vip_status","vip_top20pct_by_spend"] if c in enriched.columns]
print("\nPreview (first 10 rows):")
print(enriched.head(10)[peek_cols].to_string(index=False))


Loading: data_warehouse\processed_sales_data.csv
VIP cluster chosen: 1
Written:
 - data_warehouse\vip_sales_data.csv 
 - reverse_etl\vip_customers.csv

Preview (first 10 rows):
customer_id product_id                 sale_date  total_revenue vip_status  vip_top20pct_by_spend
      C_001       P101 2023-01-15 00:00:00+00:00          300.0    Non-VIP                      0
      C_002       P102 2023-01-20 00:00:00+00:00          225.0    Non-VIP                      0
      C_003       P101 2023-01-02 00:00:00+00:00          600.0    Non-VIP                      0
      C_004       P104 2023-05-02 00:00:00+00:00           30.0    Non-VIP                      0
      C_005       P102 2023-10-02 00:00:00+00:00          150.0    Non-VIP                      0
      C_006       P101 2023-02-20 00:00:00+00:00          450.0    Non-VIP                      0
      C_008       P104 2023-01-03 00:00:00+00:00          150.0    Non-VIP                      0
      C_009       P102 2023-05-03 00:00